In [1]:
import numpy as np

In [2]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jamiewelsh2/rap-lyrics")

print("Path to dataset files:", path)


# Load the dataset
df = pd.read_csv(path + "/updated_rappers.csv")
display(df.head())

Path to dataset files: C:\Users\Nima\.cache\kagglehub\datasets\jamiewelsh2\rap-lyrics\versions\1


,Unnamed: 0,artist,song,lyric,next lyric
0,0,Fetty Wap,Trap Queen,rgf productions,remy boyz yahah
1,0,Fetty Wap,Trap Queen,remy boyz yahah,1738 ayy
2,0,Fetty Wap,Trap Queen,1738 ayy,im like hey whats up hello
3,0,Fetty Wap,Trap Queen,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,0,Fetty Wap,Trap Queen,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [3]:
# Load data
def load_data(path):
    df = pd.read_csv(path + "/updated_rappers.csv")
    df = df[['lyric', 'next lyric']].dropna()
    return df

df = load_data(path)
display(df.head())

,lyric,next lyric
0,rgf productions,remy boyz yahah
1,remy boyz yahah,1738 ayy
2,1738 ayy,im like hey whats up hello
3,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [5]:
import tensorflow as tf
import numpy as np
import math


# Data preprocessing: tekst tokenizen, vocabulaire bouwen en tokens maken.
def preprocess_text(text):
    # Verander de tekst naar kleine letters en splitst op spaties.
    words = text.lower().split()
    vocab = sorted(set(words))
    word2idx = {word: i for i, word in enumerate(vocab)}
    idx2word = {i: word for i, word in enumerate(vocab)}
    tokens = np.array([word2idx[word] for word in words], dtype=np.int32)
    return tokens, word2idx, idx2word

# Een generator om batches te maken voor training.
def create_batches(tokens, batch_size, seq_length):
    n_batches = len(tokens) // (batch_size * seq_length)
    tokens = tokens[:n_batches * batch_size * seq_length]
    x = tokens.reshape((batch_size, -1))
    for i in range(0, x.shape[1] - seq_length, seq_length):
        x_batch = x[:, i:i+seq_length]
        y_batch = x[:, i+1:i+seq_length+1]
        yield x_batch, y_batch

Deze implementatie van de Mamba class is geïnspireerd op de Mamba-paper. De recurrente update in __call__
gebruikt het selectie-mechanisme dat ervoor zorgt dat het model inputafhankelijk kan bepalen wat relevant is.
Dit is een vereenvoudigde versie waarin we de kernideeën van de paper toepassen:
- Een inputembedding-laag,
- Een selectie-mechanisme (vergelijkbaar met een gating-methode in RNNs) zoals afgeleid in Theorem 1,
- Een outputprojectie voor next-token predictie.

Daarnaast toont de generate-methode hoe je autoregressief kunt genereren, te beginnen vanaf een startwoord.
De functie retourneert de gegenereerde tekst samen met het aantal woorden.

In [ ]:
# De Mamba class gebaseerd op de Mamba-paper.
# Deze implementatie gebruikt een selectie-mechanisme zoals beschreven in Theorem 1:
# gₜ = σ(W_gate · xₜ + b_gate) en hₜ = (1 - gₜ) · hₜ₋₁ + gₜ · xₜ.
# Hiermee wordt de recurrente update gemodelleerd op basis van de input, wat overeenkomt met het idee van input-afhankelijke selectie.
class Mamba(tf.Module):
    def __init__(self, vocab_size, embed_dim, name=None):
        super(Mamba, self).__init__(name=name)
        # Embedding matrix: zet tokenindices om in vectoren.
        self.embedding = tf.Variable(tf.random.uniform([vocab_size, embed_dim], -0.1, 0.1), name="embedding")
        # Gate parameters voor het selectie-mechanisme.
        self.W_gate = tf.Variable(tf.random.uniform([embed_dim, embed_dim], -0.1, 0.1), name="W_gate")
        self.b_gate = tf.Variable(tf.zeros([embed_dim]), name="b_gate")
        # Output projectie naar de vocabulaire-dimensie.
        self.W_out = tf.Variable(tf.random.uniform([embed_dim, vocab_size], -0.1, 0.1), name="W_out")
        self.b_out = tf.Variable(tf.zeros([vocab_size]), name="b_out")
        
    def __call__(self, inputs, initial_state=None):
        # inputs: shape (batch_size, seq_length) met token indices.
        # initial_state: shape (batch_size, embed_dim); als None, dan wordt de state op nul gezet.
        batch_size = tf.shape(inputs)[0]
        seq_length = tf.shape(inputs)[1]
        x_embed = tf.nn.embedding_lookup(self.embedding, inputs)  # (batch_size, seq_length, embed_dim)
        
        if initial_state is None:
            h = tf.zeros([batch_size, x_embed.shape[-1]], dtype=tf.float32)
        else:
            h = initial_state
        
        outputs = []
        # Itereer over de tijdstappen.
        for t in range(seq_length):
            x_t = x_embed[:, t, :]  # (batch_size, embed_dim)
            # Bereken de gate: gₜ = σ(W_gate·xₜ + b_gate)
            g_t = tf.nn.sigmoid(tf.matmul(x_t, self.W_gate) + self.b_gate)
            # Update de hidden state volgens: hₜ = (1 - gₜ)*hₜ₋₁ + gₜ*xₜ
            h = (1 - g_t) * h + g_t * x_t
            # Bereken de output logits op basis van de hidden state.
            logits = tf.matmul(h, self.W_out) + self.b_out
            outputs.append(logits)
        # Zet de outputs samen: (batch_size, seq_length, vocab_size)
        logits_seq = tf.stack(outputs, axis=1)
        return logits_seq, h
    
    def generate(self, start_word, word2idx, idx2word, num_words, temperature=1.0):
        # Genereer een tekstreeks op basis van een startwoord.
        current_word = start_word.lower()
        generated = [current_word]
        # Initialiseer de state.
        h = tf.zeros([1, self.embedding.shape[1]], dtype=tf.float32)
        token = word2idx.get(current_word, 0)
        for _ in range(num_words - 1):
            x_t = tf.nn.embedding_lookup(self.embedding, [token])  # (1, embed_dim)
            g_t = tf.nn.sigmoid(tf.matmul(x_t, self.W_gate) + self.b_gate)  # (1, embed_dim)
            h = (1 - g_t) * h + g_t * x_t
            logits = tf.matmul(h, self.W_out) + self.b_out  # (1, vocab_size)
            logits = logits / temperature
            prob = tf.nn.softmax(logits)
            # Sample de volgende token.
            token = tf.random.categorical(tf.math.log(prob), num_samples=1)[0, 0].numpy()
            word = idx2word[token]
            generated.append(word)
        return ' '.join(generated), len(generated)




In [17]:
# training_data
training_text = df['lyric'].str.cat(sep=' ')

sample_text = training_text[:50000]


In [18]:

# Data preprocessing.
tokens, word2idx, idx2word = preprocess_text(sample_text)
vocab_size = len(word2idx)
embed_dim = 32 

# Maak een instantie van het Mamba-model.
model = Mamba(vocab_size, embed_dim)

# Training parameters.
batch_size = 2
seq_length = 5
learning_rate = 0.01
optimizer = tf.optimizers.Adam(learning_rate)

# Eenvoudige training loop op het voorbeeldcorpus (voor demonstratiedoeleinden).
for epoch in range(10):
    total_loss = 0
    count = 0
    for x_batch, y_batch in create_batches(tokens, batch_size, seq_length):
        with tf.GradientTape() as tape:
            logits, _ = model(x_batch)
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_batch, logits=logits))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        total_loss += loss.numpy()
        count += 1
    print("Epoch", epoch+1, "Loss:", total_loss/count)

# Genereer tekst vanaf een startwoord.
start_word = "pretty"
generated_text, word_count = model.generate(start_word, word2idx, idx2word, num_words=15)
print("Gegenereerde tekst:", generated_text)
print("Aantal woorden:", word_count)


Epoch 1 Loss: 5.32694960512766
Epoch 2 Loss: 3.754908847708192
Epoch 3 Loss: 3.021244940681708
Epoch 4 Loss: 2.5827971611444545
Epoch 5 Loss: 2.299780155310152
Epoch 6 Loss: 2.099918206311469
Epoch 7 Loss: 1.9537975243144226
Epoch 8 Loss: 1.8381781202756693
Epoch 9 Loss: 1.755425330265434
Epoch 10 Loss: 1.687303255960401
Gegenereerde tekst: pretty eyes so damn robin wings bitch his inspiration put in work for that payoff
Aantal woorden: 15
